In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow import keras

In [2]:
model= keras.models.load_model('Drowsy1.h5')

In [3]:
def face_coordinates(landmarks) :
    '''This function takes the face landmarks of one person in one frame and  
    returns the (x,y) coordinates of face in 2D and (x,y,z) coordinates in 3D.
    '''
    
    face_3d=[]
    face_2d=[]
    for idx, landmark in enumerate(landmarks):
        ''' idx runs over all the 468 face landmark points '''
        
        x,y= int(landmark.x * width), int(landmark.y * height)
        face_2d.append([x,y])
        face_3d.append([x,y,landmark.z])
    face_2d = np.array(face_2d, dtype=np.float64)
    face_3d = np.array(face_3d, dtype=np.float64)
    return face_2d, face_3d

In [4]:
def head_angles(face_2d, face_3d):
    ''' This function takes the 2D and 3D face landmarks coordinates and
         returns the x, y, z angles of the head with respect to camera.'''
    
    focal_length = 2* width
    cam_matrix = np.array([[focal_length, 0, height/2], [0,focal_length, width/2], [0,0,1]])
    dist_matrix = np.zeros((4,1), dtype=np.float64) # distortion parameters

    success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix) # Rotation and translation

    rot_matrix, jacobian= cv2.Rodrigues(rot_vec)
    angles, mtxr, mtxq, qx,qy,qz = cv2.RQDecomp3x3(rot_matrix)
    x= angles[0] * 360
    y= angles[1] * 360
    z= angles[2] * 360
    return (x, y, z)

In [5]:
def drowsy_detector(image):
    image = cv2.resize(image1, (224, 224))
    image = image/255.
    image = np.reshape(image, (1,224,224,3))
    pred = np.argmax(model.predict(image))     
    return pred

In [6]:
def head_direction(x,y):
    ''' This function takes the x and y angles
    and gives the direction of head turn.'''
    
    if y < 2:
        text = 'Looking Left'
    elif y >14:
        text= 'Looking Right'
    elif x < -5:
        text = 'Looking Down'
    elif x > 5:
        text = 'Looking Up'
    else: text = 'Looking Forward'
    return text

In [7]:
cap = cv2.VideoCapture(0)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence =0.5)
counter_head =1
counter_alert=1
from pygame import mixer
mixer.init()
voice_head = mixer.Sound('forward.wav')
voice_alert= mixer.Sound('alert.wav')

while True:
    still_reading, frame = cap.read()
    if not still_reading:
        break
    else:
        image1 = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image1)
        height, width, channels = image1.shape
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                face_2d, face_3d = face_coordinates(face_landmarks.landmark)
                x, y, z = head_angles(face_2d, face_3d)
                direction = head_direction(x,y)
                if direction != 'Looking Forward':
                    counter_head+=1
                cv2.putText(image1, direction, (20,150), cv2.FONT_HERSHEY_COMPLEX, 2, (0,255,0), 2)
                
            prediction = drowsy_detector(image1)
            if (prediction==0 or prediction ==3):
                message='The person is drowsy'
                counter_alert +=1
                cv2.putText(image1, message, (50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 2)
            else:
                message = 'Not Drowsy'
                cv2.putText(image1, message, (50,50),cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0), 2)
            
        else:
            print('No face detected')
            continue
        if counter_head % 5 ==0:
            voice_head.play()
        if counter_alert % 5==0:
            voice_alert.play()

    image = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)    
    cv2.imshow('Drowsiness Detection', image)
    if cv2.waitKey(1000) & 0xFF ==ord('k'):
        break
        
cap.release()
cv2.destroyAllWindows()

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
{'Closed': 0, 'Open': 1, 'no_yawn': 2, 'yawn': 3}